In [ ]:
# Global Variables
seed = random.seed(123)
number_clusters = 3

# Analisis Exploratorio

In [ ]:
# Variable classification
train_data = pd.read_csv('./data/train.csv', encoding = "ISO-8859-1")
test_data = pd.read_csv('./data/test.csv', encoding = "ISO-8859-1")
variables = pd.read_csv('./data/variables.txt', encoding = "ISO-8859-1")
quant_vars = list(variables.loc[(variables['Clasification'] == 'Cuantitativa')]['Variable'].values)
quali_vars = list(variables.loc[(variables['Clasification'] == 'Cualitativa')]['Variable'].values)[1:]

### Analizando las variables numericas

In [ ]:
train_data[quant_vars].describe()

In [ ]:
for var in quant_vars:
    data = train_data[var].dropna(how='all', axis=0)
    
    # Gráfico
    sns.displot(data, kde=True)

    # Mostrando normalidad
    print('\033[1m' + var + '\033[0m' + ': Kurtosis:', stats.kurtosis(data), 'Skewness:', stats.skew(data), '\n')

### Analizando las variables categoricas

In [ ]:
for var in quali_vars:
  plt.figure(figsize=(20,5))
  train_data[var].value_counts().plot(kind='bar')
  plt.show()

### Analizando la variable de interes

In [ ]:
#skewness and kurtosis
print('Skewness: %f' % train_data['SalePrice'].skew())
print('Kurtosis: %f' % train_data['SalePrice'].kurt())
print('\n---Describe---')
train_data['SalePrice'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.65, 0.7, 0.8, 0.9, 0.95])

In [ ]:
stat,p = stats.shapiro(train_data[["SalePrice"]].dropna())
print('Kolmogorov-Smirnov:\np=%f\n'% p)
ks_statistic, p_value = diag.lilliefors(train_data[["SalePrice"]].dropna())
print('Lilliefors:\nks=%f\np=%f'%(ks_statistic,p_value))

In [ ]:
sns.displot(train_data['SalePrice'], kde=True)

### Correlacion

In [ ]:
k = 10 #number of variables for heatmap
corrmat = train_data.corr()
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

### Obteniendo la relacion entre las variables mas significativas

In [ ]:
sns.pairplot(train_data[cols],hue="SalePrice")
plt.show()
# quant_vars

# Analizando data

In [ ]:
def categorize(row, option): 
    if (option == 0):
        if row['SalePrice'] > 0 and row['SalePrice'] <= 179280:
            return 1
        return 0
    elif (option == 1):
        if (row['SalePrice'] > 179280 and row['SalePrice'] < 326100):
            return 1
        return 0
    elif (option == 2):
        if (row['SalePrice'] >= 326100):
            return 1
        return 0

train_data['isMedium'] = train_data.apply(lambda row: categorize(row, 1), axis=1)
train_data['isExpensive'] = train_data.apply(lambda row: categorize(row, 2), axis=1)

train_data

In [ ]:
# isLow isMedium isExpensive
result = 'isMedium'

In [ ]:
copied_train_data = train_data.copy()
copied_train_data = copied_train_data.fillna(0)


y = copied_train_data.pop(result) #La variable respuesta
X = copied_train_data[quant_vars] #El resto de los datos

70% de entrenamiento y 30% prueba

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7, random_state=42)

## Creando el modelo

In [ ]:
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
cm = confusion_matrix(y_test,y_pred)

#### Resultados esperados

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
precision =precision_score(y_test, y_pred,average='micro')
recall =  recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test,y_pred,average='micro')
print('Matriz de confusión para detectar virginica\n',cm)
print('Accuracy: ',accuracy)

In [ ]:
# evaluate on the train dataset
cross_validate(logReg, X_train, y_train, return_train_score=True)

## Grafica

In [ ]:
sns.regplot(x=X_test["SalePrice"], y=y_pred, data=train_data, logistic=True, ci=None)